# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>
    
#### 1. What test is appropriate for this problem? Does CLT apply?

We could use **two sampled z or t-test**, but because we are dealing with a large sample size, we must use **two sampled z-statistic test**

**CLT conditions**:

Randomization: experiment information explicit that randomly assigning identical résumés black-sounding or white-sounding names.

Independent: Sampling without replacement requires using a sample size n <= 10% of the population. Clearly, 4870 people are less than 10% of the population (black and white people in the American labor market)

Sample size condition: we have 392 callbacks between white and black sounding names. 235 of that belongs to white and 157 belongs to black, so we have more than 10 in each case.

Then, we can apply CLT

#### 2. What are the null and alternate hypotheses?
H0: There is not significant difference between callbacks for labor interviews received between white and black people in the American labor market.

H1: There is a significant difference.
  
</p>
</div>

In [5]:
# number total of callbacks for white and black-sounding names
print (sum(data[data.race=='b'].call) + sum(data[data.race=='w'].call))

392.0


3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.


In [6]:
call_back_w = sum(data[data.race=='w'].call)
n_w = len(data[data.race=='w'])
p_w = call_back_w/n_w

In [7]:
call_back_b = sum(data[data.race=='b'].call)
n_b = len(data[data.race=='b'])
p_b = call_back_b/n_b

In [8]:
p_hat = (call_back_w + call_back_b)/(n_w + n_b)

In [9]:
z_stat = ((p_w-p_b)-0)/np.sqrt(p_hat*(1-p_hat)*((1/n_w)+(1/n_b)))

In [10]:
p = (1 - stats.norm.cdf(abs(z_stat)))*2

In [11]:
print ('z-statistics = %.5f, p = %.10f' %(z_stat, p))

z-statistics = 4.10841, p = 0.0000398389


In [12]:
if p >= 0.05:
    print ('Fail to reject the null hypothesis')
else:
    print ('Reject the null hypothesis')

Reject the null hypothesis


Using a *confidence level* of 0.05, we reject the null hypothesis. Then, **there is a significance difference in callbacks to interview between black and white people.**

In [13]:
z_critical = stats.norm.ppf(q = 0.975) 
margin = z_critical*np.sqrt(p_hat*(1-p_hat)*((1/n_w)+(1/n_b)))
print ('Margin of error = %.5f, confidence_interval = [%.5f, %.5f]' %(margin,p_hat - margin, p_hat + margin ))


Margin of error = 0.01528, confidence_interval = [0.06521, 0.09577]


**95% chace** that true difference between proportions are between 0.06521 and 0.09577. It means that it's not null, firming up that there is a significant difference.

Now, we use bootstrap replicates:

In [14]:
def bootstrap_replicate_1d(data):
    return sum(np.random.choice(data, size=len(data)))

In [15]:
def draw_bs_reps(data, size=1):
    bs_replicates = np.empty(size)
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data)/len(data)
    return bs_replicates

In [16]:
bs_replicates_w = draw_bs_reps(data[data.race=='w'].call, 1000)
bs_replicates_b = draw_bs_reps(data[data.race=='w'].call, 1000)

bs_diff = bs_replicates_w - bs_replicates_b

In [17]:
p_bs = 1 - np.sum((abs(bs_diff) > 0 )/len(bs_diff))
print ('p = %.10f' %(p_bs))

p = 0.0230000000


In [18]:
if p >= 0.05:
    print ('Fail to reject the null hypothesis')
else:
    print ('Reject the null hypothesis')

Reject the null hypothesis


This result is interesting, because using a confidence level of 0.05, we reject the null hypothesis. However, if we choose another common confidence level (0.01) we must fail to reject the null hypothesis. Doing the t-statistic test, we got a p << 0.05, so we dont have dude to reject the hypothesis. 

In this case, using a confidence level of 0.05, we reject the null hypothesis so there is significance difference between the samples proportion according to the t-stastistic and bootstrapping test.

<div class="span5 alert alert-success">
<p> 
    
**4. Write a story describing the statistical significance in the context or the original problem.**
    
**5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?**

How we could determine if there is racial discrimination in the American job marker? 

We have analyzed a dataset with postulations to jobs randomly assigning identical résumés black-sounding or white-sounding names and observing the impact on requests for interviews from employers. 
    
We used the column with 'w' and 'b' to indicate if the names sounds like if they belongs to black and white people and the column that indicates whether the resume received a call from employers or not.
    
Bootstrapping replicates and two sampled z-statistics approaches show that there is significant difference between both groups. In both case we reject the null hypothesis getting p < 0.05.
    
This analysis suggests that race/names has influence in callbacks, but we don't have evidence to assert that is the most important factor to determine the success or fail in a labor search. </p>
</div>